In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas_ta as ta
import re
import string
import nltk
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis.sklearn
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from IPython.core.display import clear_output
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from keras import callbacks
from IPython.core.display import clear_output
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.tag import pos_tag_sents
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import gensim
from gensim import corpora
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

/Users/ryanshintaku/opt/anaconda3/lib/python3.8/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters
/var/folders/qr/c62bp79j3c91gdkr6wydmq7h0000gn/T/ipykernel_2437/750470391.py:18: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output
/var/folders/qr/c62bp79j3c91gdkr6wydmq7h0000gn/T/ipykernel_2437/750470391.py:29: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output
/Users/ryanshintaku/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used

In [2]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

2022-05-10 12:42:10.312473: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [4]:
articles = pd.read_csv( "full_article2.txt", index_col=None, header=None, engine='python')
articles.columns = ['Symbol', 'Article', 'Month-Day', 'Year']
articles['Year'] = articles['Year'].astype(str)
articles['Date'] = articles['Month-Day'] + " " + articles['Year']
articles['Date'] = articles['Date'].str.strip()
articles['Date'] = articles['Date'].str.slice(start = 2, stop = 13)
articles['Date'] = articles['Date'].str.strip()
articles = articles[['Symbol', 'Article', 'Date']]
articles

,Symbol,Article,Date
0,MO,The other big story Monday was oil storming h...,Mar 22 2022
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,Mar 22 2022
2,MO,The members of this alcohol and tobacco indus...,Mar 17 2022
3,MO,(Reuters) - An administrative law judge dismi...,Feb 24 2022
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,Feb 15 2022
...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,Jun 25 2015
62475,ZTS,Investing.com – U.S. stocks were lower after ...,Jun 25 2015
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,Apr 27 2015
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,Mar 09 2015


In [5]:
articles.at[articles[articles['Date'].str.contains('hours') == True].index, 'Date'] = 'Apr 06 2022'
articles = articles[articles['Date'].str.len() == 11]
articles = articles[articles['Date'].str.contains("minutes") == False]
articles['Date'] = pd.to_datetime(articles['Date'], format = '%b %d %Y')
articles

,Symbol,Article,Date
0,MO,The other big story Monday was oil storming h...,2022-03-22
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,2022-03-22
2,MO,The members of this alcohol and tobacco indus...,2022-03-17
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15
...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09


In [6]:
df = articles.copy()
df

,Symbol,Article,Date
0,MO,The other big story Monday was oil storming h...,2022-03-22
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,2022-03-22
2,MO,The members of this alcohol and tobacco indus...,2022-03-17
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15
...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09


In [7]:
daily_prices = pd.read_csv('daily_prices.csv')
daily_prices

,Date,Open,High,Low,Close,Volume,Symbol
0,2014-04-09,107.283329,107.974706,106.885983,107.950867,2671900.0,MMM
1,2014-04-10,107.863458,108.419740,106.409171,106.464806,2857200.0,MMM
2,2014-04-11,105.988008,106.361514,105.209213,105.209213,2368200.0,MMM
3,2014-04-14,106.051558,106.107193,104.915156,105.813148,2058100.0,MMM
4,2014-04-15,105.908540,106.766808,104.915176,106.560181,2119000.0,MMM
...,...,...,...,...,...,...,...
1000210,2022-04-04,191.139999,191.979996,187.179993,188.740005,2603600.0,ZTS
1000211,2022-04-05,188.740005,190.740005,187.690002,188.399994,2059300.0,ZTS
1000212,2022-04-06,188.160004,192.330002,186.520004,191.380005,2843500.0,ZTS
1000213,2022-04-07,191.199997,199.339996,190.970001,197.929993,2781400.0,ZTS


In [8]:
current_price = []
for i in range(len(df)):
    if i % 500 == 0:
        print(i)
    if i % 2000 == 0:
        clear_output(wait = True)
    subset = daily_prices[(daily_prices['Date'] <= str(df.iloc[i]['Date'])[0: 10]) & (df.iloc[i]['Symbol'] == daily_prices['Symbol'])]
    if len(subset) == 0:
        current_price.append(np.nan)
        continue
    subset = subset[subset['Date'] == max(subset['Date'])]
    current_price.append(subset.iloc[0]['Close'])
current_price[0]

52.40848159790039

In [12]:
def get_current_price(date, sym):
    subset = daily_prices[(daily_prices['Date'] <= str(date)[0: 10]) & (sym == daily_prices['Symbol'])]
    if len(subset) == 0:
        return np.nan
    subset = subset[subset['Date'] == max(subset['Date'])]
    return subset.iloc[0]['Close']

In [9]:
# 30 days from current date
next_price2 = []
for i in range(len(df)):
    if i % 500 == 0:
        print(i)
    if i % 2000 == 0:
        clear_output(wait = True)
    next_date = df.iloc[i]['Date'] + pd.DateOffset(days = 30)
    next_date = str(next_date)[0:10]
    subset = daily_prices[(daily_prices['Date'] <= next_date) & (df.iloc[i]['Symbol'] == daily_prices['Symbol'])]
    if len(subset) == 0:
        next_price2.append(np.nan)
        continue
    subset = subset[subset['Date'] == max(subset['Date'])]
    if subset.iloc[0]['Date'] <= str(df.iloc[i]['Date'] + pd.DateOffset(days = 20))[0:10]:
        next_price2.append(np.nan)
        continue
    next_price2.append(subset.iloc[0]['Close'])
next_price2[0]

nan

In [10]:
df['currentPrice'] = current_price
df['nextMonthPrice'] = next_price2
df['dateSold'] = df['Date'] + pd.DateOffset(days = 30)
df

,Symbol,Article,Date,currentPrice,nextMonthPrice,dateSold
0,MO,The other big story Monday was oil storming h...,2022-03-22,52.408482,NaN,2022-04-21
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,2022-03-22,52.408482,NaN,2022-04-21
2,MO,The members of this alcohol and tobacco indus...,2022-03-17,50.815872,54.110001,2022-04-16
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24,48.731728,53.619999,2022-03-26
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15,49.469044,50.815872,2022-03-17
...,...,...,...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25,52.983288,47.348190,2015-07-25
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25,52.983288,47.348190,2015-07-25
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27,44.803303,47.309917,2015-05-27
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09,44.207355,44.210152,2015-04-08


In [11]:
df = df.dropna()
#df = df[df['Date'] <= '2022-03-01']
df

,Symbol,Article,Date,currentPrice,nextMonthPrice,dateSold
2,MO,The members of this alcohol and tobacco indus...,2022-03-17,50.815872,54.110001,2022-04-16
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24,48.731728,53.619999,2022-03-26
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15,49.469044,50.815872,2022-03-17
5,MO,Investing.com - Altria (NYSE:MO) reported on ...,2022-01-27,49.488705,50.609425,2022-02-26
6,MO,By Geoffrey Smith Investing.com -- Global mar...,2022-01-27,49.488705,50.609425,2022-02-26
...,...,...,...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25,52.983288,47.348190,2015-07-25
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25,52.983288,47.348190,2015-07-25
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27,44.803303,47.309917,2015-05-27
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09,44.207355,44.210152,2015-04-08


In [12]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    sentence = []
    for word, tag in tokens:
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v' 
        else:
            pos = 'a'
        sentence.append(lemmatizer.lemmatize(word, pos))
    return sentence

In [13]:
df['title/no_stopwords'] = df.apply(lambda row: pos_tag(word_tokenize(row['Article'])), axis = 1)
df['title/no_stopwords'] = df.apply(lambda row: lemmatize_sentence(row['title/no_stopwords']), axis = 1)
df

/var/folders/qr/c62bp79j3c91gdkr6wydmq7h0000gn/T/ipykernel_2437/2665988695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title/no_stopwords'] = df.apply(lambda row: pos_tag(word_tokenize(row['Article'])), axis = 1)
/var/folders/qr/c62bp79j3c91gdkr6wydmq7h0000gn/T/ipykernel_2437/2665988695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title/no_stopwords'] = df.apply(lambda row: lemmatize_sentence(row['title/no_stopwords']), axis = 1)


,Symbol,Article,Date,currentPrice,nextMonthPrice,dateSold,title/no_stopwords
2,MO,The members of this alcohol and tobacco indus...,2022-03-17,50.815872,54.110001,2022-04-16,"[The, member, of, this, alcohol, and, tobacco,..."
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24,48.731728,53.619999,2022-03-26,"[(, Reuters, ), -, An, administrative, law, ju..."
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15,49.469044,50.815872,2022-03-17,"[WASHINGTON, (, Reuters, ), -A, judge, at, the..."
5,MO,Investing.com - Altria (NYSE:MO) reported on ...,2022-01-27,49.488705,50.609425,2022-02-26,"[Investing.com, -, Altria, (, NYSE, :, MO, ), ..."
6,MO,By Geoffrey Smith Investing.com -- Global mar...,2022-01-27,49.488705,50.609425,2022-02-26,"[By, Geoffrey, Smith, Investing.com, --, Globa..."
...,...,...,...,...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25,52.983288,47.348190,2015-07-25,"[(, Reuters, ), -, Canada-based, drugmaker, Va..."
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25,52.983288,47.348190,2015-07-25,"[Investing.com, –, U.S., stock, be, low, after..."
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27,44.803303,47.309917,2015-05-27,"[By, David, RandallNEW, YORK, (, Reuters, ), -..."
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09,44.207355,44.210152,2015-04-08,"[By, Svea, Herbst-BaylissBOSTON, (, Reuters, )..."


In [14]:
def remove_noise(reviews, stop_words = ()):

    cleaned_tokens = []

    for token in reviews:
        token = token.strip()
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        token = token.replace('--', '')
        token = token.replace('-', '')
        token = token.replace('–', '')
        token = token.replace('investing.com', '')
        token = token.replace('``', '')
        token = token.replace("'s", '')
        token = token.replace('..', '')
        token = token.replace('...', '')
        token = token.replace("'", '')
        token = token.replace("’",'')
        token = token.replace("reuters", "")
        token = token.replace("say", "")
        token = token.replace("company", "")
        token = token.replace("inc", "")
        token = token.replace('p', '')
        token = token.replace('apos', '')
        token = token.replace('aposs', '')
        token = token.replace('er', '')
        token = token.replace('erformer', '')
        token = token.replace('eole', '')
        token = token.replace('ale', '')
        token = token.replace('oint', '')
        token = token.replace('reort', '')
        token = token.replace('aoss', '')
        token = token.replace('also', '')

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

<>:7: DeprecationWarning: invalid escape sequence \(
<>:7: DeprecationWarning: invalid escape sequence \(
/var/folders/qr/c62bp79j3c91gdkr6wydmq7h0000gn/T/ipykernel_2437/374916687.py:7: DeprecationWarning: invalid escape sequence \(
  token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\


In [15]:
stop_words = list(stopwords.words('english'))
df['title/no_stopwords'] = df.apply(lambda row: remove_noise(row['title/no_stopwords'], stop_words), axis = 1)
df

/var/folders/qr/c62bp79j3c91gdkr6wydmq7h0000gn/T/ipykernel_2437/992663860.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title/no_stopwords'] = df.apply(lambda row: remove_noise(row['title/no_stopwords'], stop_words), axis = 1)


,Symbol,Article,Date,currentPrice,nextMonthPrice,dateSold,title/no_stopwords
2,MO,The members of this alcohol and tobacco indus...,2022-03-17,50.815872,54.110001,2022-04-16,"[memb, alcohol, tobacco, industrybacked, marij..."
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24,48.731728,53.619999,2022-03-26,"[reuts, administrative, law, judge, dismiss, f..."
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15,49.469044,50.815872,2022-03-17,"[washington, reuts, judge, u.s., fedal, trade,..."
5,MO,Investing.com - Altria (NYSE:MO) reported on ...,2022-01-27,49.488705,50.609425,2022-02-26,"[investing.com, altria, nyse, mo, thursday, fo..."
6,MO,By Geoffrey Smith Investing.com -- Global mar...,2022-01-27,49.488705,50.609425,2022-02-26,"[geoffrey, smith, investing.com, global, marke..."
...,...,...,...,...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25,52.983288,47.348190,2015-07-25,"[reuts, canadabased, drugmak, vant, pharmaceut..."
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25,52.983288,47.348190,2015-07-25,"[investing.com, u.s., stock, low, aft, close, ..."
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27,44.803303,47.309917,2015-05-27,"[david, randallnew, york, reuts, nasdaq, comos..."
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09,44.207355,44.210152,2015-04-08,"[svea, hbstbaylissboston, reuts, billionaire, ..."


In [16]:
df['title/clean'] = df.apply(lambda row: ' '.join(row['title/no_stopwords']), axis = 1)
df

/var/folders/qr/c62bp79j3c91gdkr6wydmq7h0000gn/T/ipykernel_2437/3931287753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title/clean'] = df.apply(lambda row: ' '.join(row['title/no_stopwords']), axis = 1)


,Symbol,Article,Date,currentPrice,nextMonthPrice,dateSold,title/no_stopwords,title/clean
2,MO,The members of this alcohol and tobacco indus...,2022-03-17,50.815872,54.110001,2022-04-16,"[memb, alcohol, tobacco, industrybacked, marij...",memb alcohol tobacco industrybacked marijuana ...
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24,48.731728,53.619999,2022-03-26,"[reuts, administrative, law, judge, dismiss, f...",reuts administrative law judge dismiss fedal t...
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15,49.469044,50.815872,2022-03-17,"[washington, reuts, judge, u.s., fedal, trade,...",washington reuts judge u.s. fedal trade commis...
5,MO,Investing.com - Altria (NYSE:MO) reported on ...,2022-01-27,49.488705,50.609425,2022-02-26,"[investing.com, altria, nyse, mo, thursday, fo...",investing.com altria nyse mo thursday fourth q...
6,MO,By Geoffrey Smith Investing.com -- Global mar...,2022-01-27,49.488705,50.609425,2022-02-26,"[geoffrey, smith, investing.com, global, marke...",geoffrey smith investing.com global market ste...
...,...,...,...,...,...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25,52.983288,47.348190,2015-07-25,"[reuts, canadabased, drugmak, vant, pharmaceut...",reuts canadabased drugmak vant pharmaceuticals...
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25,52.983288,47.348190,2015-07-25,"[investing.com, u.s., stock, low, aft, close, ...",investing.com u.s. stock low aft close thursda...
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27,44.803303,47.309917,2015-05-27,"[david, randallnew, york, reuts, nasdaq, comos...",david randallnew york reuts nasdaq comosite la...
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09,44.207355,44.210152,2015-04-08,"[svea, hbstbaylissboston, reuts, billionaire, ...",svea hbstbaylissboston reuts billionaire inves...


In [29]:
!pip install -q -U "tensorflow-text==2.8.*"

In [30]:
!pip install -q tf-models-official==2.7.0

In [34]:
!pip install tensorflow-text

In [17]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [18]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [19]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [20]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [21]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='linear', name="output")(l)

In [22]:
def build_linear_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='linear', name='regressor')(net)
    return tf.keras.Model(text_input, net)

In [23]:
def build_numerical_model():
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(1001, )),
      tf.keras.layers.Dense(18, activation='relu'),
      tf.keras.layers.Dense(18, activation='relu'),
        tf.keras.layers.Dense(9, activation='relu'),
        tf.keras.layers.Dense(9, activation='relu')
    ])
    return model

In [60]:
text_test = ['this is such an amazing movie!']
linear_model = build_linear_model()
linear_model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [49]:
date_split = '2018-12-30'
X_train = df[df['Date'] < date_split]['title/clean']
y_train = df[df['Date'] < date_split]['nextMonthPrice']
X_test = df[df['Date'] >= date_split]['title/clean']
y_test = df[df['Date'] >= date_split]['nextMonthPrice']

In [65]:
linear_model.fit(X_train, y_train, epochs = 5)

Epoch 1/5
739/739 [==============================] - 1966s 3s/step - loss: 113879.7891 - mean_absolute_error: 179.7513
Epoch 2/5
739/739 [==============================] - 2037s 3s/step - loss: 114471.8984 - mean_absolute_error: 177.7278
Epoch 3/5
739/739 [==============================] - 1935s 3s/step - loss: 113895.7109 - mean_absolute_error: 179.6014
Epoch 4/5
739/739 [==============================] - 1862s 3s/step - loss: 113887.3125 - mean_absolute_error: 178.9717
Epoch 5/5
739/739 [==============================] - 1837s 2s/step - loss: 113901.1953 - mean_absolute_error: 179.7078


In [76]:
init_investment = 10000
predicted = pd.DataFrame(df.loc[X_test.index]['currentPrice'])
predicted['predictedPrice'] = linear_model.predict(X_test)
predicted['percentChange'] = ((predicted['predictedPrice'] - predicted['currentPrice']) / predicted['currentPrice']) * 100
predicted['actualPrice'] = y_test
predicted['articleDate'] = df.loc[X_test.index]['Date']
predicted['Symbol'] = df.loc[X_test.index]['Symbol']
predicted = predicted.drop_duplicates()
predicted = predicted[predicted['articleDate'] <= '2022-03-01']
predicted

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


,currentPrice,predictedPrice,percentChange,actualPrice,articleDate,Symbol
3,48.731728,166.831039,242.345834,53.619999,2022-02-24,MO
4,49.469044,166.831039,237.243308,50.815872,2022-02-15,MO
5,49.488705,166.831039,237.109327,50.609425,2022-01-27,MO
7,43.000420,166.831039,287.975375,42.190182,2021-10-28,MO
9,46.405338,166.831055,259.508326,42.518135,2021-10-26,MO
...,...,...,...,...,...,...
62459,197.935364,166.831055,-15.714377,201.654922,2021-09-17,ZTS
62460,202.981216,166.831055,-17.809609,199.830048,2021-09-14,ZTS
62461,201.315887,166.831039,-17.129720,209.104050,2021-08-06,ZTS
62462,202.133591,166.831039,-17.464960,203.250473,2021-07-30,ZTS


In [84]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
model.fit(X_train, y_train, epochs=2)

In [24]:
vectorizer = TfidfVectorizer(max_features = 500, min_df = 10, max_df = .9, stop_words=stopwords.words('english'))
tfidf = vectorizer.fit_transform(df['title/clean'].tolist())
tfidf = tfidf.toarray()
tfidf

array([[0.        , 0.        , 0.        , ..., 0.07255649, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05415784, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.02883584, ..., 0.10456525, 0.        ,
        0.02752557],
       [0.        , 0.        , 0.        , ..., 0.07900149, 0.        ,
        0.06932069],
       [0.11004132, 0.        , 0.        , ..., 0.        , 0.        ,
        0.09922076]])

In [25]:
tfidf_pd = pd.DataFrame(tfidf)
tfidf_pd.index = df.index
tfidf_pd['Symbol'] = df['Symbol']
tfidf_pd['currentDate'] = df['Date']
tfidf_pd['nextDate'] = df['dateSold']
tfidf_pd['currentPrice'] = df['currentPrice']
tfidf_pd['nextMonthPrice'] = df['nextMonthPrice']
tfidf_pd

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,Symbol,currentDate,nextDate,currentPrice,nextMonthPrice
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.072556,0.000000,0.000000,MO,2022-03-17,2022-04-16,50.815872,54.110001
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,MO,2022-02-24,2022-03-26,48.731728,53.619999
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.087893,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.054158,0.000000,0.000000,MO,2022-02-15,2022-03-17,49.469044,50.815872
5,0.000000,0.000000,0.000000,0.074378,0.000000,0.000000,0.070818,0.000000,0.000000,0.074378,...,0.000000,0.000000,0.079431,0.000000,0.000000,MO,2022-01-27,2022-02-26,49.488705,50.609425
6,0.000000,0.000000,0.063274,0.042970,0.000000,0.074474,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.026812,0.114723,0.084822,0.030200,MO,2022-01-27,2022-02-26,49.488705,50.609425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.098142,0.000000,0.000000,...,0.081675,0.064148,0.054896,0.000000,0.072253,ZTS,2015-06-25,2015-07-25,52.983288,47.348190
62475,0.128434,0.040425,0.000000,0.000000,0.096599,0.031731,0.000000,0.069911,0.130064,0.036617,...,0.058181,0.022848,0.000000,0.000000,0.077203,ZTS,2015-06-25,2015-07-25,52.983288,47.348190
62476,0.000000,0.000000,0.028836,0.000000,0.000000,0.033940,0.000000,0.000000,0.034779,0.039165,...,0.000000,0.000000,0.104565,0.000000,0.027526,ZTS,2015-04-27,2015-05-27,44.803303,47.309917
62477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.094159,0.000000,0.098634,...,0.000000,0.123089,0.079001,0.000000,0.069321,ZTS,2015-03-09,2015-04-08,44.207355,44.210152


In [107]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1001, )),
  tf.keras.layers.Dense(18, activation='relu'),
  tf.keras.layers.Dense(18, activation='relu'),
    tf.keras.layers.Dense(9, activation='relu'),
    tf.keras.layers.Dense(9, activation='relu'),
    tf.keras.layers.Dense(1, activation = 'linear')
])

In [134]:
date_split = '2018-12-30'
X_train2 = tfidf_pd[tfidf_pd['currentDate'] < date_split].drop(['currentDate', 'Symbol', 'nextMonthPrice', 'nextDate'], axis = 1)
y_train2 = tfidf_pd[tfidf_pd['currentDate'] < date_split]['nextMonthPrice']
X_test2 = tfidf_pd[tfidf_pd['currentDate'] >= date_split].drop(['currentDate', 'Symbol', 'nextMonthPrice', 'nextDate'], axis = 1)
y_test2 = tfidf_pd[tfidf_pd['currentDate'] >= date_split]['nextMonthPrice']
X_train2

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,currentPrice
90,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.110872,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.063414,41.797832
91,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.062471,0.031740,0.0,0.0,0.027139,0.000000,0.0,0.035731,41.797832
92,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.073244,0.000000,0.0,0.0,0.031819,0.000000,0.0,0.041892,41.797832
93,0.185238,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.098815,...,0.000000,0.097325,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.055666,41.797832
94,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.172343,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.098573,41.967552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.056013,0.0,0.0,0.047894,0.000000,0.0,0.063056,52.983288
62475,0.107506,0.040629,0.000000,0.076676,0.040363,0.033757,0.0,0.041351,0.040559,0.000000,...,0.000000,0.000000,0.019132,0.0,0.0,0.000000,0.000000,0.0,0.064613,52.983288
62476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.039899,0.041849,0.000000,0.0,0.0,0.090901,0.039548,0.0,0.023936,44.803303
62477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.111033,0.0,0.0,0.071204,0.000000,0.0,0.062497,44.207355


In [108]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(X_train, y_train, epochs = 10, batch_size = 100)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
237/237 [==============================] - 2s 2ms/step - loss: 141247.0156 - mean_absolute_error: 165.4578
Epoch 2/10
237/237 [==============================] - 0s 2ms/step - loss: 37893.0039 - mean_absolute_error: 57.0808
Epoch 3/10
237/237 [==============================] - 0s 2ms/step - loss: 869.4555 - mean_absolute_error: 11.5306
Epoch 4/10
237/237 [==============================] - 1s 2ms/step - loss: 859.3608 - mean_absolute_error: 10.2833
Epoch 5/10
237/237 [==============================] - 0s 2ms/step - loss: 864.5341 - mean_absolute_error: 9.9513
Epoch 6/10
237/237 [==============================] - 1s 2ms/step - loss: 864.6549 - mean_abs

In [109]:
predicted = pd.DataFrame(tfidf_pd.loc[X_test.index]['currentPrice'])
predicted['predictedPrice'] = model.predict(X_test)
predicted['percentChange'] = ((predicted['predictedPrice'] - predicted['currentPrice']) / predicted['currentPrice']) * 100
predicted['actualPrice'] = y_test
predicted['articleDate'] = tfidf_pd.loc[X_test.index]['currentDate']
predicted['Symbol'] = tfidf_pd.loc[X_test.index]['Symbol']
predicted = predicted.drop_duplicates()
predicted = predicted[predicted['articleDate'] <= '2022-03-01']
predicted

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


,currentPrice,predictedPrice,percentChange,actualPrice,articleDate,Symbol
3,48.731728,48.012875,-1.475123,53.619999,2022-02-24,MO
4,49.469044,50.319611,1.719392,50.815872,2022-02-15,MO
5,49.488705,49.697136,0.421169,50.609425,2022-01-27,MO
6,49.488705,49.043571,-0.899464,50.609425,2022-01-27,MO
7,43.000420,42.426373,-1.334980,42.190182,2021-10-28,MO
...,...,...,...,...,...,...
62459,197.935364,199.818863,0.951573,201.654922,2021-09-17,ZTS
62460,202.981216,203.996445,0.500159,199.830048,2021-09-14,ZTS
62461,201.315887,202.315796,0.496686,209.104050,2021-08-06,ZTS
62462,202.133591,204.011429,0.929008,203.250473,2021-07-30,ZTS


In [110]:
predicted[predicted['percentChange'] > 10]

,currentPrice,predictedPrice,percentChange,actualPrice,articleDate,Symbol
4108,11.955596,13.516169,13.053073,11.502626,2021-11-08,AMCR
4109,11.436213,13.128057,14.793734,12.131990,2021-10-13,AMCR
4125,16.290001,19.180231,17.742357,18.250000,2022-03-01,AAL
4129,16.959999,19.594101,15.531262,17.010000,2022-02-24,AAL
4131,18.840000,22.824762,21.150542,16.549999,2022-02-15,AAL
...,...,...,...,...,...,...
61548,18.109463,20.078457,10.872736,16.740021,2020-05-29,WMB
61602,16.637434,18.315117,10.083784,12.631417,2020-02-25,WMB
61669,19.170204,21.107004,10.103179,20.500780,2019-12-02,WMB
61727,20.997965,23.668375,12.717471,19.629269,2019-09-13,WMB


In [26]:
m1 = build_linear_model()
m2 = build_numerical_model()

In [27]:
combinedInput = concatenate([m1.output, m2.output])
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

In [28]:
model = Model(inputs=[m1.input, m2.input], outputs=x)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
model.fit(x=[X_train, X_train2], y=y_train,epochs=5, batch_size=125)

Epoch 1/5
  1/190 [..............................] - ETA: 1:09:00 - loss: 182908.1406 - mean_absolute_error: 179.2379

In [36]:
df

,Symbol,Article,Date,currentPrice,nextMonthPrice,dateSold,title/no_stopwords,title/clean
2,MO,The members of this alcohol and tobacco indus...,2022-03-17,50.815872,54.110001,2022-04-16,"[memb, alcohol, tobacco, industrybacked, marij...",memb alcohol tobacco industrybacked marijuana ...
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24,48.731728,53.619999,2022-03-26,"[reuts, administrative, law, judge, dismiss, f...",reuts administrative law judge dismiss fedal t...
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15,49.469044,50.815872,2022-03-17,"[washington, reuts, judge, u.s., fedal, trade,...",washington reuts judge u.s. fedal trade commis...
5,MO,Investing.com - Altria (NYSE:MO) reported on ...,2022-01-27,49.488705,50.609425,2022-02-26,"[investing.com, altria, nyse, mo, thursday, fo...",investing.com altria nyse mo thursday fourth q...
6,MO,By Geoffrey Smith Investing.com -- Global mar...,2022-01-27,49.488705,50.609425,2022-02-26,"[geoffrey, smith, investing.com, global, marke...",geoffrey smith investing.com global market ste...
...,...,...,...,...,...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25,52.983288,47.348190,2015-07-25,"[reuts, canadabased, drugmak, vant, pharmaceut...",reuts canadabased drugmak vant pharmaceuticals...
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25,52.983288,47.348190,2015-07-25,"[investing.com, u.s., stock, low, aft, close, ...",investing.com u.s. stock low aft close thursda...
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27,44.803303,47.309917,2015-05-27,"[david, randallnew, york, reuts, nasdaq, comos...",david randallnew york reuts nasdaq comosite la...
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09,44.207355,44.210152,2015-04-08,"[svea, hbstbaylissboston, reuts, billionaire, ...",svea hbstbaylissboston reuts billionaire inves...


In [40]:
df[['Symbol', 'Date', 'dateSold', 'currentPrice', 'nextMonthPrice']].to_csv('bert_df.csv')

In [51]:
from official.nlp import optimization
epochs = 5
steps_per_epoch = len(X_train)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [53]:
from transformers import CamembertTokenizer

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
encoded_corpus = tokenizer(text=df['Article'].tolist(),
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=300,
                            return_attention_mask=True)
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

In [55]:
def filter_long_descriptions(tokenizer, descriptions, max_len):
    indices = []
    lengths = tokenizer(descriptions, padding=False, 
                     truncation=False, return_length=True)['length']
    for i in range(len(descriptions)):
        if lengths[i] <= max_len-2:
            indices.append(i)
    return indices
short_descriptions = filter_long_descriptions(tokenizer, 
                               df['Article'].tolist(), 300)
input_ids = np.array(input_ids)[short_descriptions]
attention_mask = np.array(attention_mask)[short_descriptions]
labels = df.prix.to_numpy()[short_descriptions]

Token indices sequence length is longer than the specified maximum sequence length for this model (589 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [ ]:
#https://f35.csc.calpoly.edu:8000/user/rtshinta/lab?